# HANK with Investment

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt   
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
plt.rcParams.update({"axes.grid" : True, "grid.color": "black", "grid.alpha":"0.25", "grid.linestyle": "--"})
plt.rcParams.update({'font.size': 14})
from HANKModel import HANKModelClass
import numba as nb 

## 1. <a id='toc1_'></a>[Setup](#toc0_)

Create HANK model 

In [ ]:
model = HANKModelClass(name='baseline')
model.find_ss(do_print=False)
model._compute_jac_hh()
model.calc_MPC()

In [ ]:
model.test_path()

Create RANK model by using RA household jacobians instead of HA household jacobians 

In [ ]:
def create_RANK_model(model_HA):
    model_RA = model_HA.copy()
    model_RA.par.HH_type = 'RANK'

    # Get Jacobian of RA model 
    M_RA = model_RA.get_RA_J()

    # overwrite HH jacobian such that GEModelTools uses RA jacobian 
    # Note that if you recompute the HH jacobian (model.compute_jac_hh() or model.compute_jacs with the keyword "skip_hh=False") then this gets overwritten
    # and the model is effectively HANK again 
    
    for o in ['C_hh', 'A_hh']: # outputs    
        for i in ['Z', 'ra', 'chi']: # inputs 
            model_RA.jac_hh[(o,i)] = M_RA[o][i]

    return model_RA

model_RA = create_RANK_model(model)